# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

path = ("../output_data/city_weather.csv")
df = pd.read_csv(path)
df.head()

,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed
0,0,cidreira,55.44,-30.1811,-50.2056,93.0,10.0,8.03
1,1,arraial do cabo,67.93,-22.9661,-42.0278,76.0,74.0,16.58
2,2,pangnirtung,51.30,66.1451,-65.7125,75.0,99.0,3.87
3,3,zhigansk,61.12,66.7697,123.3711,54.0,56.0,11.77
4,4,taolanaro,0.00,0.0000,0.0000,0.0,0.0,0.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = df[["latitude", "longitude"]]
humidity = df["humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
hotel_df = pd.DataFrame(df[(df['temp(f)']<80) & (df['temp(f)']>70) & (df['humidity']<45)])

hotel_df

,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed
74,74,komsomolskiy,76.42,40.4272,71.7189,28.0,48.0,2.30
83,83,hamilton,78.84,39.1834,-84.5333,44.0,1.0,8.05
119,119,sarkand,74.43,45.4100,79.9186,28.0,100.0,6.49
164,164,konya,74.68,38.1667,32.5000,24.0,0.0,5.95
193,193,corning,76.42,42.1428,-77.0547,40.0,1.0,0.00
239,239,guiratinga,77.31,-16.3494,-53.7583,27.0,66.0,3.62
254,254,sibay,71.40,52.7181,58.6658,40.0,55.0,3.67
271,271,shahreza,72.57,32.0339,51.8794,26.0,11.0,5.19
300,300,hearst,71.62,49.6835,-83.6665,29.0,19.0,4.52
302,302,urumqi,74.35,43.8010,87.6005,27.0,0.0,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df['Hotel Name'] = ""
#hotel_df['City'] = ""
#hotel_df['Country'] = ""
hotel_df['compound_code'] = ""
hotel_df

,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed,Hotel Name,compound_code
74,74,komsomolskiy,76.42,40.4272,71.7189,28.0,48.0,2.30,,
83,83,hamilton,78.84,39.1834,-84.5333,44.0,1.0,8.05,,
119,119,sarkand,74.43,45.4100,79.9186,28.0,100.0,6.49,,
164,164,konya,74.68,38.1667,32.5000,24.0,0.0,5.95,,
193,193,corning,76.42,42.1428,-77.0547,40.0,1.0,0.00,,
239,239,guiratinga,77.31,-16.3494,-53.7583,27.0,66.0,3.62,,
254,254,sibay,71.40,52.7181,58.6658,40.0,55.0,3.67,,
271,271,shahreza,72.57,32.0339,51.8794,26.0,11.0,5.19,,
300,300,hearst,71.62,49.6835,-83.6665,29.0,19.0,4.52,,
302,302,urumqi,74.35,43.8010,87.6005,27.0,0.0,4.47,,


In [21]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "42.1428,-77.0547"
    
    
}

    # Use the search term: "International Airport" and our lat/lng
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
name_address = requests.get(base_url, params=params).json()

name_address["results"][0]["name"]
name_address["results"][0]["plus_code"]["compound_code"]
name_address["results"][0]        


{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 42.1532559, 'lng': -77.08044129999999},
  'viewport': {'northeast': {'lat': 42.1545602802915,
    'lng': -77.0789468697085},
   'southwest': {'lat': 42.1518623197085, 'lng': -77.08164483029151}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'icon_background_color': '#909CE1',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
 'name': 'Fairfield Inn by Marriott Corning Riverside',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 1366,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109857128391989750920">Fairfield Inn by Marriott Corning Riverside</a>'],
   'photo_reference': 'Aap_uECJfSVRefMp8IZG5WT4dA4j9USqTE57mV8Ygep-zz1_HRZ_mUaOKpZ2WUb4DoVjnvKbo3nSOPD0Wvr5zQKPov2RRG0Iet_90jvSaVIMlwwi5gERBKPC4cFU1JD-xwx5euJSC3mLKQgofKeee4SoSM9KtYC9bCecuoYh-CLxGovlsRW8',
   'width': 2048}],
 'place_id': 'ChIJj7xF0

In [16]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
    
    
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "compound_code"] = name_address["results"][0]["plus_code"]["compound_code"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)   
hotel_df['compound_code'].replace('', np.nan, inplace=True)   
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.dropna(subset=['compound_code'], inplace=True)
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed,Hotel Name,compound_code
83,83,hamilton,78.84,39.1834,-84.5333,44.0,1.0,8.05,Six Acres Bed & Breakfast,"5FP4+97 Cincinnati, OH, USA"
119,119,sarkand,74.43,45.4100,79.9186,28.0,100.0,6.49,Гостиница Сарканд,"CW58+9Q Sarķand, Kazakhstan"
164,164,konya,74.68,38.1667,32.5000,24.0,0.0,5.95,Bahcesaray Koyu Muhtarligi,"5F25+VJ Sarayönü/Konya, Turkey"
193,193,corning,76.42,42.1428,-77.0547,40.0,1.0,0.00,Fairfield Inn by Marriott Corning Riverside,"5W39+8R Corning, NY, USA"
239,239,guiratinga,77.31,-16.3494,-53.7583,27.0,66.0,3.62,Hotel Angra do Cerrado - Guiratinga MT,"J6XQ+CV Centro, Guiratinga - MT, Brazil"
254,254,sibay,71.40,52.7181,58.6658,40.0,55.0,3.67,Hotel Tatiana,"PM95+MC Sibay, Republic of Bashkortostan, Russia"
271,271,shahreza,72.57,32.0339,51.8794,26.0,11.0,5.19,خوابگاه,"2V7F+9H Shahreza, Isfahan Province, Iran"
300,300,hearst,71.62,49.6835,-83.6665,29.0,19.0,4.52,Companion,"M8RM+37 Hearst, ON, Canada"
302,302,urumqi,74.35,43.8010,87.6005,27.0,0.0,4.47,Hua Ling Grand Hotel Urumqi,"QHWJ+33 Shuimogou District, Ürümqi, Xinjiang, ..."
386,386,mashhad,75.34,36.2970,59.6062,33.0,0.0,4.61,Khanye Sabz Hotel,"8H3X+5W Mashhad, Razavi Khorasan Province, Iran"


In [17]:
hotel_df[["compound_code","City","Country"]] = hotel_df["compound_code"].str.split(expand=True)
hotel_df['City'] = hotel_df['City'].str.rstrip(',')

hotel_df

ValueError: Columns must be same length as key

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [ ]:
# Add marker layer ontop of heat map

fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_template)
fig.add_layer(markers)

# Display figure
fig